In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-3.4.2-bin-hadoop3'

In [3]:
from pyspark.sql import SparkSession

In [5]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [7]:
# Initialize Spark session

sc = SparkSession.builder.appName("Product_Recommendation") \
.config ("spark.sql.shuffle.partitions", "16") \
.config("spark.driver.maxResultSize","4g") \
.config ("spark.sql.execution.arrow.enabled", "true") \
.config("spark.driver.memory", "4g") \
.config("spark.executor.cores", "4") \
.getOrCreate()

sc.sparkContext.setLogLevel("ERROR")

In [8]:
df = sc.read.option('header', True).csv(r"C:\Users\sonal\OneDrive\Desktop\Project\Group06\2020-Apr.csv")

In [9]:
from pyspark.sql.functions import *
df = df.withColumn('event_time', to_timestamp('event_time'))
df = df.withColumn('date', date_format("event_time", "yyyy-MM-dd"))

In [11]:
# Print the number of interactions between users and products, sorted by carts

interactions = df.groupby(['user_id', 'product_id']).agg(sum(when(df['event_type'] == 'view', 1)).alias('views'),
                                                         sum(when(df['event_type'] == 'cart', 1)).alias('carts'),
                                                         sum(when(df['event_type'] == 'purchase', 1)).alias('purchases'))

interactions.sort('carts', ascending=False).show()

+---------+----------+-----+-----+---------+
|  user_id|product_id|views|carts|purchases|
+---------+----------+-----+-----+---------+
|561804169|   1005119| null|  705|     null|
|515823933|   4804056|  268|  357|       57|
|627964145|   1002544|  386|  305|      186|
|636877416|   1004836|    1|  275|        1|
|637820403|   1005267|   18|  225|     null|
|553146361| 100068493|    1|  224|     null|
|553750625|   1004838|   29|  219|        6|
|629017322|   1004226|    9|  180|     null|
|602700164| 100068493|  181|  178|       88|
|515015932|   2702331|  314|  169|      116|
|526141229|   1307555|   57|  166|       20|
|532696490| 100068488|    3|  164|        1|
|518514888|   1005105|  240|  158|       86|
|640826351|  12707969|    4|  148|        1|
|642438807| 100055405|   11|  144|     null|
|621593683|   1002544|   12|  141|        5|
|641599967|   1801966|   27|  140|        7|
|619714299| 100003951|   22|  138|       10|
|563544259|   3601074|   10|  135|        1|
|608251121

In [12]:
def preprocess(df):
    
    # Change data types
    df = df.withColumn('event_time', to_timestamp('event_time'))
    df = df.withColumn('user_id', col('user_id').cast('integer'))
    df = df.withColumn('product_id', col('product_id').cast('integer'))
    df = df.withColumn('category_id', col('category_id').cast('long'))
    
    # Limit the number of carts to 1 per session for each user-product pair
    cart_df = df.filter(col('event_type') == 'cart')
    df = df.filter(col('event_type') != 'cart')
    cart_df = cart_df.dropDuplicates(subset=['product_id', 'user_id', 'user_session'])
    df = df.union(cart_df)
    
    # Split category codes into sub categories
    '''
    from pyspark.sql.functions import split

    df = (df
      .withColumn('Category', split(df['category_code'], "\\.").getItem(0))
      .withColumn('Sub_category', split(df['category_code'], "\\.").getItem(1))
      .withColumn('Product', split(df['category_code'], "\\.").getItem(2)))
    '''
    return df


In [13]:
df = preprocess(df)

In [14]:
df.show(2)

+-------------------+----------+----------+-------------------+--------------------+-------+-------+---------+--------------------+----------+
|         event_time|event_type|product_id|        category_id|       category_code|  brand|  price|  user_id|        user_session|      date|
+-------------------+----------+----------+-------------------+--------------------+-------+-------+---------+--------------------+----------+
|2020-04-01 05:30:00|      view|   1201465|2232732101407408685|apparel.shoes.sli...|samsung| 230.38|568984877|e2456cef-2d4f-42b...|2020-04-01|
|2020-04-01 05:30:01|      view|   1307156|2053013554658804075|electronics.audio...|  apple|1352.67|514955500|38f43134-de83-471...|2020-04-01|
+-------------------+----------+----------+-------------------+--------------------+-------+-------+---------+--------------------+----------+
only showing top 2 rows



In [21]:
# Get the timestamp of the most recent event in the df
last_date = df.agg(max('event_time')).collect()[0][0]
df = df.withColumn('last_date', lit(last_date))

# Calculate the recency of each event in terms of days
df = df.withColumn('recency', (col('last_date').cast('double') - col('event_time').cast('double')) / 86400)
df = df.drop('last_date')

# Half-life decay function, the value of an event is halved after 20 days
df = df.withColumn('recency_coef', expr('exp(ln(0.5)*recency/20)'))

In [23]:
# Find the number of views, carts and purchases for each user-product pair
interactions = df.groupby(['user_id', 'product_id']).agg(sum(when(df['event_type'] == 'view', 1) * df['recency_coef']).alias('views'),
                                                         sum(when(df['event_type'] == 'cart', 1) * df['recency_coef']).alias('carts'),
                                                         sum(when(df['event_type'] == 'purchase', 1) * df['recency_coef']).alias('purchases'))
interactions = interactions.na.fill(0)

In [25]:
def calculate_interaction_matrix(df, view_weight=0.1, cart_weight=0.3, purchase_weight=1.0):
 
    # Create a new column with the weighted interaction value
    df = df.withColumn('interaction', view_weight * col('views') + cart_weight * col('carts') + purchase_weight * col('purchases'))
    
    # Use log10 value for views, carts and purchases
    df = df.withColumn('interaction', log10(col('interaction') + 1))
    
    # Set the max possible value to 100 (log100 = 2)
    df = df.withColumn('interaction', when(col('interaction') > 2, 2).otherwise(col('interaction')))

    return df

In [27]:
interaction_matrix = calculate_interaction_matrix(interactions)

In [29]:
#  User-product pairs with the highest interaction scores

interaction_matrix.sort('interaction', ascending=False).show()

+---------+----------+------------------+------------------+------------------+------------------+
|  user_id|product_id|             views|             carts|         purchases|       interaction|
+---------+----------+------------------+------------------+------------------+------------------+
|582249321|  17303060| 151.2180522911544|  70.0345391662787| 64.82335487004885|               2.0|
|522016074|   1004246| 91.25545831347586| 50.85077913559578| 80.96545496214094|               2.0|
|515015932|   2702331| 229.7294101275016|46.970533624974735| 84.96658287445932|               2.0|
|627964145|   1002544|231.81214397099683|  85.2906523862065|111.82412526370732|               2.0|
|637890453|   1004625| 982.2671413260865|               0.0|               0.0|1.9966286100423212|
|638901021| 100068488| 129.7541250342781|49.259348232049256| 61.74705040774978|1.9566498623193596|
|518514888|   1005105|153.96318975415417|57.433089339174025|55.487773065298995| 1.949946029817058|
|648484033

In [30]:
interaction_matrix_1K = interaction_matrix.limit(1000)

In [41]:
interaction_matrix_1K.show(3)

+---------+----------+-------------------+------------------+------------------+--------------------+
|  user_id|product_id|              views|             carts|         purchases|         interaction|
+---------+----------+-------------------+------------------+------------------+--------------------+
|568984877|   1201465|0.35355353241305676|               0.0|               0.0|0.015089433333691357|
|633281427| 100077607|  5.625789535792961|0.4788379411010751|0.4788446637645328|  0.3394663481190854|
|548207294|   1005105| 2.5146706936722643|               0.0|               0.0| 0.09741942623328653|
+---------+----------+-------------------+------------------+------------------+--------------------+
only showing top 3 rows



In [33]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

def simple_als(interaction_matrix):
    
    # Train-test split
    (train, test) = interaction_matrix.randomSplit([0.8, 0.2])
    
    # Initialize the model with the optimized parameters
    als = ALS(userCol='user_id', itemCol='product_id', ratingCol='interaction', 
              alpha=1, regParam=0.005, rank=15, implicitPrefs=True, 
              nonnegative=True, coldStartStrategy='drop')

    # Fit the ALS model on the ratings data
    model = als.fit(train)
    
    # Make predictions
    predictions = model.transform(test)
    # Calculate the RMSE and MAE metrics
    evaluator = RegressionEvaluator(metricName='rmse', labelCol='interaction', predictionCol='prediction')
    rmse = evaluator.evaluate(predictions)
    mae = evaluator.setMetricName('mae').evaluate(predictions)
    print('test rmse:' + str(rmse) + ' mae:' + str(mae))
    
    return model

In [35]:
als_model = simple_als(interaction_matrix_1K)

test rmse:0.08876871352724139 mae:0.06204417585520686


In [47]:
# 3 random users
user_subset = [568984877, 633281427, 548207294]

# Recommend top 2 products for the users
recommendations = sc.createDataFrame([(user, 0) for user in user_subset], ['user_id', 'product_id'])
recommendations = als_model.recommendForUserSubset(recommendations, 2)

In [49]:
recommendations.show()

Py4JJavaError: An error occurred while calling o1086.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 654.0 failed 1 times, most recent failure: Lost task 1.0 in stage 654.0 (TID 1885) (localhost executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:601)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:583)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:772)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:749)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:514)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at sc

In [ ]:
# Select user 564068124

recs_for_user_1 = sc.createDataFrame(recommendations.collect()[1][1])

In [ ]:
# Products with the highest number of interactions for the user

interactions.filter(col('user_id') == 564068124).sort('purchases', ascending=False).show()